# From poll to Index


## Setup

In [ ]:
import os 
import sys 
import json 
import random
from matplotlib import pyplot
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [ ]:
spark = (
    SparkSession
        .builder
        .appName('poll_to_idxs')
        .getOrCreate()
    )

spark

## CSV loading and cleaning

In [ ]:
#Directory where to access the poll
poll_dir = "../data/poll_data.csv"

# Load the csv poll as a dataframe
df = (
    spark.read.format("csv")
    # .option("encoding", "UTF-8")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(poll_dir)
)

# List of columns that will not be considered individually scored (due to not be used in the calculation)
skip_cols = [
    "Ho letto e accettato l'informativa e confermo inoltre di avere più di 18 anni",
    "Informazioni cronologiche",
    "Quanti anni hai?",
    "Genere",
    "Da quante persone è composto il tuo nucleo familiare?",
    "Occupazione",
    "Quanto è grande la tua azienda?",
    "Da che regione provieni?",
    "Provincia di provenienza",
    "In che regione lavori/studi?",
    "Provincia del luogo di lavoro/studio",
    "Invalidità",
    "Tipo di residenza",
    "Numero di persone con cui convivi",
    "Entrate Familiari Mensili Nette",
    "Entrate Personali Mensili Nette ",
]

# Load the questions json
with open("../data/questions.json", "r", encoding="utf-8") as questions_file:
    questions = json.load(questions_file)

# Create a question_text:question_idx map, useful to alias the columns
questions_idxs = {questions[q]["question_text"]: q for q in questions.keys()}

# Alias the columns (long column names cause bugs and are hard to use)
for k, v in questions_idxs.items():
    df = df.withColumnRenamed(k, v)

df.show(5)

## Computing the scores

In [ ]:
# "Subtract" the list of unscorable cols from the list of scorable cols
scorable_cols = [item for item in df.columns if item not in skip_cols]

# Iterate over the scorable columns to create the scores dictionary
scores = {}
for col in scorable_cols:
    # Shortcuts for some useful values
    question_type = questions[col]["question_type"]
    question_score = questions[col]["question_score"]
    question_answers = questions[col]["answers"]

    # Skip unscored questions
    if len(question_answers) == 0 and question_type != "comma_separated":
        continue

    # Collect the Row values for the current column and access its value (index 0)
    answers = [str(d[0]) for d in df.select(col).collect()]

    # Handle score computation differently based on question_type
    row_scores = []
    if question_type in ["basic", "multivalue"]:
        for indexes_cols_df in answers:
            row_scores.append(question_answers[indexes_cols_df]["answer_score"] * question_score)

        scores[col] = row_scores

    elif question_type == "comma_separated":
        for i in answers:
            row_scores.append(len(i.split(";")))

        scores[col] = row_scores

In [ ]:
# Create the scores dataframe from the scores dictionary
df_scores = spark.createDataFrame(pd.DataFrame(scores))
df_scores.show(5)

In [ ]:
# The indexes dictionary, which will be used to compute the indexes dataframe
indexes_cols = {}

## Computing the indexes

### Purchases indexes

In [ ]:
# Purchases indexes calculation
df_s = df_scores.select([f"S_{d}" for d in range(1, 10)]).collect()

# i_S_col
i_S_col = []
for s in df_s:
    i_S_col.append(sum([val for val in s]) / 1450)
indexes_cols["i_S"] = i_S_col


col_S_1 = df.select("S_1").collect()
S_1 = []
answer1 = questions["S_1"]["answers"]
for i in range(len(col_S_1)):
    s1 = col_S_1[i][0]
    for j in answer1.keys():
        if s1 == j:
            val = answer1[j]["answer_score_1"]
            S_1.append(val)
            break

col_S_2 = df.select("S_2").collect()
S_2 = []
answer2 = questions["S_2"]["answers"]
for i in range(len(col_S_2)):
    s2 = str(col_S_2[i][0])
    for j in answer2.keys():
        if s2 == j:
            val = answer2[j]["answer_score_1"]
            S_2.append(val)
            break
        
df_s1 = df_scores.select([f"S_{d}" for d in range(3, 10)]).collect()
i_tot_S_col = []
k = 0  # variabile temporanea per accedere agli elementi di S_1 ed S_2 ed effettuare il rapporto
for s_1 in df_s1:
    somma = sum([val1 for val1 in s_1])
    r = S_2[k] / S_1[k]
    i_tot_S_col.append(somma * r)
    k += 1
indexes_cols["i_tot_S"] = i_tot_S_col

In [ ]:
# Calcolo indici domande relative al Vestiario
df_v = df_scores.select([f"V_{d}" for d in range(10, 13)]).collect()

i_V_col = []
for v in df_v:
    i_V_col.append(sum([val for val in v]) / 550)

indexes_cols["i_V"] = i_V_col

col_V_10 = df.select("V_10").collect()
V_10 = []
answer10 = questions["V_10"]["answers"]
for i in range(len(col_V_10)):
    v10 = str(col_V_10[i][0])
    for j in answer10.keys():
        if v10 == j:
            val = answer10[j]["answer_score_1"]
            V_10.append(val)
            break
df_v1 = df_scores.select([f"V_{d}" for d in range(11, 13)]).collect()
i_tot_V_col = []
l = 0  # variabile temporanea per accedere agli elementi di V_10
for v1 in df_v1:
    somma1 = sum([val1 for val1 in v1]) / 400
    i_tot_V_col.append(somma1 * V_10[l])
    l += 1
indexes_cols["i_tot_V"] = i_tot_V_col

In [ ]:
# Calcolo indici domande relative alla Casa
df_c = df_scores.select([f"C_{d}" for d in range(13, 15)]).collect()

i_C_col = []
for c in df_c:
    i_C_col.append(sum([val for val in c]) / 350)

indexes_cols["i_C"] = i_C_col

i_tot_C_col = []
for c1 in df_c:
    if c1[1] == 0:
        i_tot_C_col.append(c1[0] / 150)
    else:
        i_tot_C_col.append(c1[0] / c1[1])
indexes_cols["i_tot_C"] = i_tot_C_col

In [ ]:
# Calcolo indice Educazione e Ricerca
df_e = df_scores.select([f"ER_{d}" for d in range(1, 4)]).collect()
i_ER_col = []
for e in df_e:
    i_ER_col.append(sum([val for val in e]) / 500)
indexes_cols["i_ER"] = i_ER_col

### Mobility indexes

In [ ]:
df_m1 = df_scores.select(["m_8", "m_9"]).collect()

i_m1_col = []
for s in df_m1:
    m_8, m_9 = s[0], s[1]
    i_m1_col.append(m_9 / m_8)

indexes_cols["i_M1"] = i_m1_col

In [ ]:
df_m2 = df_scores.select(["m_17", "m_18"]).collect()

i_m2_col = []
for s in df_m2:
    m_17, m_18 = s[0], s[1]
    i_m2_col.append(m_18 / m_17)

indexes_cols["i_M2"] = i_m2_col

In [ ]:
fam_size_col_label = "Da quante persone è composto il tuo nucleo familiare?"

df_m3 = df_scores.select(["m_8"]).collect()
df_fam = df.select([fam_size_col_label]).collect()

i_m3_col = []
for s in df_m3:
    m_8, f = s[0], int(df_fam[0][0])
    i_m3_col.append(m_8 / f)

indexes_cols["i_M3"] = i_m3_col

In [ ]:
df_m4 = df_scores.select(["m_11", "m_12", "m_13"]).collect()

i_m4_col = []
for s in df_m4:
    m_11, m_12, m_13 = s[0], s[1], s[2]
    i_m4_col.append(m_12 / (m_11 + m_12 + m_13))

indexes_cols["i_M4"] = i_m4_col

In [ ]:
df_m5 = df_scores.select(["m_20"]).collect()

i_m5_col = []
for s in df_m5:
    m_20 = s[0]
    i_m5_col.append(m_20)

indexes_cols["i_M5"] = i_m5_col

In [ ]:
df_m6 = df_scores.select(["m_19"]).collect()

i_m6_col = []
for s in df_m6:
    m_19 = s[0]
    i_m6_col.append(m_19)

indexes_cols["i_M6"] = i_m6_col

In [ ]:
df_m7 = df_scores.select(["m_4", "m_5"]).collect()

i_m7_col = []
for s in df_m7:
    m_4, m_5 = s[0], s[1]
    i_m7_col.append(m_5 / m_4)

indexes_cols["i_M7"] = i_m7_col

### Energy indexes

In [ ]:
# Systems (home)
df_e1 = df_scores.select(["eh2"]).collect()
i_e1_col = []
for s in df_e1:
    i_e1_col.append(s[0] / 5)
indexes_cols["i_e1"] = i_e1_col

In [ ]:
# Systems (work)
df_e2 = df_scores.select(["ew2"]).collect()
i_e2_col = []
for s in df_e2:
    i_e2_col.append(s[0] / 5)
indexes_cols["i_e2"] = i_e2_col

In [ ]:
# Sustainable Source (home)
df_e3 = df_scores.select(["eh3", "eh4"]).collect()
i_e3_col = []
for s in df_e3:
    eh3, eh4 = s[0], s[1]
    i_e3_col.append(eh3 * eh4)
indexes_cols["i_e3"] = i_e3_col

In [ ]:
# Sustainable Source (work)
df_e4 = df_scores.select(["ew3", "ew4"]).collect()
i_e4_col = []
for s in df_e4:
    ew3, ew4 = s[0], s[1]
    i_e4_col.append(ew3 * ew4)
indexes_cols["i_e4"] = i_e4_col

In [ ]:
# Efficiency Ratio
df_e5 = df_scores.select(["eh1", "eh5"]).collect()
i_e5_col = []
for s in df_e5:
    eh1, eh5 = s[0], s[1]
    i_e5_col.append(eh1 / eh5)
indexes_cols["i_e5"] = i_e5_col

In [ ]:
# GreenBuilding (home)
df_e6 = df_scores.select(["eh6"]).collect()
i_e6_col = []
for s in df_e6:
    i_e6_col.append(s[0] / 6)
indexes_cols["i_e6"] = i_e6_col

In [ ]:
# GreenBuilding (work)
df_e7 = df_scores.select(["ew5"]).collect()
i_e7_col = []
for s in df_e7:
    i_e7_col.append(s[0] / 6)
indexes_cols["i_e7"] = i_e7_col

### Water Index

In [ ]:
# Water (home)
df_water1 = df_scores.select(["wh1", "wh2", "wh3", "wh4"]).collect()
i_e8_col = []
for s in df_water1:
    wh1, wh2, wh3, wh4 = s[0], s[1], s[2], s[3]
    i_e8_col.append((wh1 + wh2 + wh3 + wh4) / 4)
indexes_cols["i_e8"] = i_e8_col

In [ ]:
# Water (work)
df_water2 = df_scores.select(["ww1", "ww2", "ww3", "ww4"]).collect()
i_e9_col = []
for s in df_water2:
    ww1, ww2, ww3, ww4 = s[0], s[1], s[2], s[3]
    i_e9_col.append((ww1 + ww2 + ww3 + ww4) / 4)
indexes_cols["i_e9"] = i_e9_col

### Waste Index

In [ ]:
# Waste (home)
df_waste1 = df_scores.select(["wasteh1", "wasteh2", "wasteh3", "wasteh4"]).collect()
i_e10_col = []
for s in df_waste1:
    wasteh1, wasteh2, wasteh3, wasteh4 = s[0], s[1], s[2], s[3]
    i_e10_col.append((wasteh1 + wasteh2 + wasteh3 + wasteh4) / 4)
indexes_cols["i_e10"] = i_e10_col

In [ ]:
# Waste (work)
df_waste2 = df_scores.select(["wastew1", "wastew2", "wastew3", "wastew4"]).collect()
i_e11_col = []
for s in df_waste1:
    wastew1, wastew2, wastew3, wastew4 = s[0], s[1], s[2], s[3]
    i_e11_col.append((wastew1 + wastew2 + wastew3 + wastew4) / 4)
indexes_cols["i_e11"] = i_e11_col

## Indexes dataframe

In [ ]:
for k in indexes_cols.keys():
    indexes_cols[k] = [round(e, 2) for e in indexes_cols[k]]

In [ ]:
indexes_cols_df = pd.DataFrame(indexes_cols)

indexes_max = {
    # "i_S": None,
    #"i_tot_S": None,
    # "i_V": None,
    # "i_C": None,
    # "i_ER": None,
    # "i_M1": 1,
    # "i_M3": None,
    # "i_M7": None,
    "i_tot_V": 5,
    "i_tot_C": 1.5,
    "i_M2": 1,
    "i_M4": 0.95,
    "i_M5": 100,
    "i_M6": 100,
    "i_e1": 1,
    "i_e2": 1,
    "i_e3": 5,
    "i_e4": 5,
    "i_e5": 1/0.015,
    "i_e6": 1,
    "i_e7": 1,
    "i_e8": 200,
    "i_e9": 200,
    "i_e10": 200,
    "i_e11": 200,
}

for c in indexes_cols_df.columns:
    val = indexes_cols_df[c] / indexes_max.get(c, indexes_cols_df[c].max())
    indexes_cols_df[c] = round(val, 3)

# create the indexes dataframe from the indexes dictionary
df_indexes = spark.createDataFrame(pd.DataFrame(indexes_cols_df))
df_indexes.show(52)

In [ ]:
df_features = df_indexes.toPandas()

vectors = []
for _, r in df_features.iterrows():
    values = []
    for i in range(len(df_features.columns)):
        values.append(r[i])

    vectors.append(Vectors.dense(values))

In [ ]:
df_vectors = spark.createDataFrame(pd.DataFrame({"features":vectors}))

## Collaborative Filtering

In [ ]:
user_id = [i for i in range(indexes_cols_df.shape[0])]

rows = []
for i in user_id:
    for index,c in enumerate(indexes_cols_df):
        rows.append((i, index, indexes_cols_df.iloc[i][index]))

cf_df = pd.DataFrame(rows, columns=["user_id", "feature_id", "value"])


cf_df = spark.createDataFrame(cf_df)
cf_df.show()

In [ ]:
ratings = cf_df
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(
    maxIter=5, 
    regParam=0.01, 
    userCol="user_id", 
    itemCol="feature_id", 
    ratingCol="value",
    coldStartStrategy="drop",
)

model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="value",
    predictionCol="prediction",
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)

In [ ]:
punctured_df = indexes_cols_df
celle_el = []
c = 1
while c <= 130:
    i = random.randint(0,51)
    j = random.randint(0,24)
    cel = (i,j)
    if cel in celle_el:
        continue
    else:
        punctured_df.iloc[i,j] = None
        celle_el.append(cel)
        c += 1
print(celle_el)
print(len(celle_el))
punctured_df

In [ ]:
user_id = [i for i in range(punctured_df.shape[0])]

rows = []
for i in user_id:
    for index,c in enumerate(punctured_df):
        rows.append((i, index, punctured_df.iloc[i][index]))

punctured_cf_df = pd.DataFrame(rows, columns=["user_id", "feature_id", "value"])


punctured_cf_df = spark.createDataFrame(punctured_cf_df)
punctured_cf_df.show(1300)

In [ ]:
ratings = punctured_cf_df
(training, test) = ratings.randomSplit([0.8, 0.2])

training.show()
print("--------------")
test.show()


FUNZIONE PER DETERMINARE COME SETTARE I PARAMETRI DEL MODELLO (????????)
def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    validation_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS().setMaxIter(maxIter).setRank(rank).setRegParam(reg)
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model
M = tune_ALS(training, test, 10, 1.0, )
print(M)




# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(
    maxIter=10, 
    regParam=1.0, 
    userCol="user_id", 
    itemCol="feature_id", 
    ratingCol="value",
    coldStartStrategy="drop"
)

model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="value",
    predictionCol="prediction",
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)


## K-means

In [ ]:
kmeans = KMeans(k=3)

kmeans.setSeed(1)
# kmeans.setWeightCol("weigh_col")
kmeans.setMaxIter(10)
kmeans.getMaxIter()
kmeans.clear(kmeans.maxIter)
kmeans.getSolver()

model = kmeans.fit(df_vectors)

model.getMaxBlockSizeInMB()
model.getDistanceMeasure()
model.setPredictionCol("prediction")
model.predict(df_vectors.head().features)

centers = model.clusterCenters()
len(centers)

transformed = model.transform(df_vectors).select("features", "prediction")
rows = transformed.collect()

## Plots

In [ ]:
sns.heatmap(df_features.corr(), cmap="crest")

In [ ]:
with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
    display(df_features.corr()[df_features.corr() >= 0.5])

In [ ]:
# df_features.plot.scatter('i_S', 'i_tot_S', alpha=0.2)
# df_features.plot.scatter('i_C', 'i_tot_C', alpha=0.2)
# df_features.plot.scatter('i_V', 'i_tot_V', alpha=0.2)
# df_features.plot.scatter('i_e8', 'i_e3', alpha=0.2)
# df_features.plot.scatter('i_e7', 'i_e6', alpha=0.2)
# df_features.plot.scatter('i_e9', 'i_e8', alpha=0.2)

#    casa  i_e_1, i_e_3, i_e_6
#  lavoro  i_e_2, i_e_4, i_e_7


scatter_matrix(df_features[['i_e1', 'i_e3', 'i_e6', 'i_e2', 'i_e4', 'i_e7']], diagonal='kde')
scatter_matrix(df_features[['i_e1', 'i_e3', 'i_e6', 'i_e2', 'i_e4', 'i_e7']], diagonal='hist')

scatter_matrix(df_features[['i_e8', 'i_e9', 'i_e10', 'i_e11']], diagonal='kde')
scatter_matrix(df_features[['i_e8', 'i_e9', 'i_e10', 'i_e11']], diagonal='hist')

scatter_matrix(df_features[['i_S', 'i_tot_S', 'i_V', 'i_tot_V', 'i_C', 'i_tot_C']], diagonal='kde')
scatter_matrix(df_features[['i_S', 'i_tot_S', 'i_V', 'i_tot_V', 'i_C', 'i_tot_C']], diagonal='hist')

In [ ]:
fig, ax = pyplot.subplots()
pyplot.xlabel('casa')
pyplot.ylabel('lavoro')
ax.scatter(df_features['i_e8'], df_features['i_e9'])

In [ ]:
print(model.hasSummary)
summary = model.summary
print(summary.k)
print(summary.clusterSizes)

summary.cluster.show(52)